In [6]:
from lambda_calculus import Variable as Var
from lambda_calculus import Abstraction as Lambda
from lambda_calculus import Application as App

from lambda_calculus.visitors.normalisation import BetaNormalisingVisitor

from lambda_calculus.terms import logic
from lambda_calculus.terms import arithmetic
from lambda_calculus.terms import pairs
from lambda_calculus.terms import combinators

# <span style="color: #A1F99C">TEST PROCEDURE</span>

In [7]:
def test_procedure(term, print_steps=True):
    beta_norm_visitor = BetaNormalisingVisitor()

    result = term
    n_steps = 0
    if print_steps:
        print(f"{n_steps}: {term}")
    for _, intermediate in term.accept(beta_norm_visitor):
        result = intermediate
        n_steps += 1
        if print_steps:
            print(f"{n_steps}: {result}")

    print(f"\n\nreduction steps: {n_steps}")
    print(f"norm term:       {result}")

# <span style="color: #A1F99C">USEFUL TERMS</span>

In [8]:
def multi_app_term(term_0, term_1, *terms):
    res_app_term = App(term_0, term_1)
    for term in terms:
        res_app_term = App(res_app_term, term)
    return res_app_term


def leq_term():
    m, n = Var("mmm"), Var("nnn")
    return Lambda("mmm", Lambda("nnn", App(
        arithmetic.ISZERO,
        App(App(arithmetic.SUBTRACT, m), n)
    )))


def eq_term():
    mm, nn = Var("mm"), Var("nn")
    return Lambda("mm", Lambda("nn", multi_app_term(
        logic.AND,
        multi_app_term(leq_term(), mm, nn),
        multi_app_term(leq_term(), nn, mm)
    )))

# <span style="color: #A1F99C">1.</span>
<span style="color:#bb91f3">(λx. (λy.y)) ((λx. (xx)) (λx. (xx)))</span> == <span style="color:#f1bd47">I</span> == <span style="color:#f1bd47">λx. x</span>
<span style="color:#c9fafa">This term reduce in one step by LO, RO strategy
And reduce in the same terms with other strategy.</span>

In [9]:
def term_test_1():
    x, y = "x", "y"
    x_, y_ = Var(x), Var(y)

    return App(
             Lambda(x, Lambda(y, y_)),
             App(
                 Lambda(x, App(x_, x_)),
                 Lambda(x, App(x_, x_))
             )
         )


test_procedure(term_test_1())

0: ((λx.(λy.y)) ((λx.(x x)) (λx.(x x))))
1: (λy.y)


reduction steps: 1
norm term:       (λy.y)


# <span style="color: #A1F99C">2.</span>
<span style="color:#bb91f3">(ITE TRUE 3 1)</span> == <span style="color:#f1bd47>">3</span> == <span style="color:#f1bd47">λx. λy. (x (x (x y)))</span>

In [10]:
def term_test_2():
    return multi_app_term(logic.IF_THEN_ELSE, logic.TRUE, arithmetic.number(3), arithmetic.number(1))


test_procedure(term_test_2())

0: ((((λp.(λa.(λb.((p a) b)))) (λx.(λy.x))) (λf.(λx.(f (f (f x)))))) (λf.(λx.(f x))))
1: (((λa.(λb.(((λx.(λy.x)) a) b))) (λf.(λx.(f (f (f x)))))) (λf.(λx.(f x))))
2: ((λb.(((λx.(λy.x)) (λf.(λx.(f (f (f x)))))) b)) (λf.(λx.(f x))))
3: (((λx.(λy.x)) (λf.(λx.(f (f (f x)))))) (λf.(λx.(f x))))
4: ((λy.(λf.(λx.(f (f (f x)))))) (λf.(λx.(f x))))
5: (λf.(λx.(f (f (f x)))))


reduction steps: 5
norm term:       (λf.(λx.(f (f (f x)))))


# <span style="color: #A1F99C">3.</span>
<span style="color:#bb91f3">(ITE FALSE 0 4)</span> == <span style="color:#f1bd47">4</span> == <span style="color:#f1bd47">λx. λy. (x (x (x (x y))))</span>

In [11]:
def term_test_3():
    return multi_app_term(logic.IF_THEN_ELSE, logic.FALSE, arithmetic.number(0), arithmetic.number(4))


test_procedure(term_test_3())

0: ((((λp.(λa.(λb.((p a) b)))) (λx.(λy.y))) (λf.(λx.x))) (λf.(λx.(f (f (f (f x)))))))
1: (((λa.(λb.(((λx.(λy.y)) a) b))) (λf.(λx.x))) (λf.(λx.(f (f (f (f x)))))))
2: ((λb.(((λx.(λy.y)) (λf.(λx.x))) b)) (λf.(λx.(f (f (f (f x)))))))
3: (((λx.(λy.y)) (λf.(λx.x))) (λf.(λx.(f (f (f (f x)))))))
4: ((λy.y) (λf.(λx.(f (f (f (f x)))))))
5: (λf.(λx.(f (f (f (f x))))))


reduction steps: 5
norm term:       (λf.(λx.(f (f (f (f x))))))


# <span style="color: #A1F99C">4.</span>
<span style="color:#bb91f3">(PRED 0)</span> == <span style="color:#f1bd47">0</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [12]:
def term_test_4():
    return multi_app_term(arithmetic.PREDECESSOR, arithmetic.number(0))


test_procedure(term_test_4())

0: ((λn.(λf.(λx.(((n (λg.(λh.(h (g f))))) (λu.x)) (λu.u))))) (λf.(λx.x)))
1: (λf.(λx.((((λf.(λx.x)) (λg.(λh.(h (g f))))) (λu.x)) (λu.u))))
2: (λf.(λx.(((λx.x) (λu.x)) (λu.u))))
3: (λf.(λx.((λu.x) (λu.u))))
4: (λf.(λx.x))


reduction steps: 4
norm term:       (λf.(λx.x))


# <span style="color: #A1F99C">5.</span>
<span style="color:#bb91f3">(PRED 5)</span> == <span style="color:#f1bd47">4</span> == <span style="color:#f1bd47">λx. λy. (x (x (x (x y))))</span>

In [13]:
def term_test_5():
    return multi_app_term(arithmetic.PREDECESSOR, arithmetic.number(5))


test_procedure(term_test_5())

0: ((λn.(λf.(λx.(((n (λg.(λh.(h (g f))))) (λu.x)) (λu.u))))) (λf.(λx.(f (f (f (f (f x))))))))
1: (λf.(λx.((((λf.(λx.(f (f (f (f (f x))))))) (λg.(λh.(h (g f))))) (λu.x)) (λu.u))))
2: (λf.(λx.(((λx.((λg.(λh.(h (g f)))) ((λg.(λh.(h (g f)))) ((λg.(λh.(h (g f)))) ((λg.(λh.(h (g f)))) ((λg.(λh.(h (g f)))) x)))))) (λu.x)) (λu.u))))
3: (λf.(λx.(((λg.(λh.(h (g f)))) ((λg.(λh.(h (g f)))) ((λg.(λh.(h (g f)))) ((λg.(λh.(h (g f)))) ((λg.(λh.(h (g f)))) (λu.x)))))) (λu.u))))
4: (λf.(λx.((λh.(h (((λg.(λh.(h (g f)))) ((λg.(λh.(h (g f)))) ((λg.(λh.(h (g f)))) ((λg.(λh.(h (g f)))) (λu.x))))) f))) (λu.u))))
5: (λf.(λx.((λu.u) (((λg.(λh.(h (g f)))) ((λg.(λh.(h (g f)))) ((λg.(λh.(h (g f)))) ((λg.(λh.(h (g f)))) (λu.x))))) f))))
6: (λf.(λx.(((λg.(λh.(h (g f)))) ((λg.(λh.(h (g f)))) ((λg.(λh.(h (g f)))) ((λg.(λh.(h (g f)))) (λu.x))))) f)))
7: (λf.(λx.((λh.(h (((λg.(λh.(h (g f)))) ((λg.(λh.(h (g f)))) ((λg.(λh.(h (g f)))) (λu.x)))) f))) f)))
8: (λf.(λx.(f (((λg.(λh.(h (g f)))) ((λg.(λh.(h (g f)))) ((λg.(λh.(h

# <span style="color: #A1F99C">6.</span>
<span style="color:#bb91f3">(PLUS 0 0)</span> == <span style="color:#f1bd47">0</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [14]:
def term_test_6():
    return multi_app_term(arithmetic.ADD, arithmetic.number(0), arithmetic.number(0))


test_procedure(term_test_6())

0: (((λm.(λn.(λf.(λx.((m f) ((n f) x)))))) (λf.(λx.x))) (λf.(λx.x)))
1: ((λn.(λf.(λx.(((λf.(λx.x)) f) ((n f) x))))) (λf.(λx.x)))
2: (λf.(λx.(((λf.(λx.x)) f) (((λf.(λx.x)) f) x))))
3: (λf.(λx.((λx.x) (((λf.(λx.x)) f) x))))
4: (λf.(λx.(((λf.(λx.x)) f) x)))
5: (λf.(λx.((λx.x) x)))
6: (λf.(λx.x))


reduction steps: 6
norm term:       (λf.(λx.x))


# <span style="color: #A1F99C">7.</span>
<span style="color:#bb91f3">(PLUS 2 3)</span> == <span style="color:#f1bd47">5</span> == <span style="color:#f1bd47">λx. λy. (x (x (x (x (x y)))))</span>

In [15]:
def term_test_7():
    return multi_app_term(arithmetic.ADD, arithmetic.number(2), arithmetic.number(3))


test_procedure(term_test_7())

0: (((λm.(λn.(λf.(λx.((m f) ((n f) x)))))) (λf.(λx.(f (f x))))) (λf.(λx.(f (f (f x))))))
1: ((λn.(λf.(λx.(((λf.(λx.(f (f x)))) f) ((n f) x))))) (λf.(λx.(f (f (f x))))))
2: (λf.(λx.(((λf.(λx.(f (f x)))) f) (((λf.(λx.(f (f (f x))))) f) x))))
3: (λf.(λx.((λx.(f (f x))) (((λf.(λx.(f (f (f x))))) f) x))))
4: (λf.(λx.(f (f (((λf.(λx.(f (f (f x))))) f) x)))))
5: (λf.(λx.(f (f ((λx.(f (f (f x)))) x)))))
6: (λf.(λx.(f (f (f (f (f x)))))))


reduction steps: 6
norm term:       (λf.(λx.(f (f (f (f (f x)))))))


# <span style="color: #A1F99C">8.</span>
<span style="color:#bb91f3">(SUBTRACT 4 1)</span> == <span style="color:#f1bd47">3</span> == <span style="color:#f1bd47">λx. λy. (x (x (x y)))</span>

In [16]:
def term_test_8():
    return multi_app_term(arithmetic.SUBTRACT, arithmetic.number(4), arithmetic.number(1))


test_procedure(term_test_8())

0: (((λm.(λn.((n (λn.(λf.(λx.(((n (λg.(λh.(h (g f))))) (λu.x)) (λu.u)))))) m))) (λf.(λx.(f (f (f (f x))))))) (λf.(λx.(f x))))
1: ((λn.((n (λn.(λf.(λx.(((n (λg.(λh.(h (g f))))) (λu.x)) (λu.u)))))) (λf.(λx.(f (f (f (f x)))))))) (λf.(λx.(f x))))
2: (((λf.(λx.(f x))) (λn.(λf.(λx.(((n (λg.(λh.(h (g f))))) (λu.x)) (λu.u)))))) (λf.(λx.(f (f (f (f x)))))))
3: ((λx.((λn.(λf.(λx.(((n (λg.(λh.(h (g f))))) (λu.x)) (λu.u))))) x)) (λf.(λx.(f (f (f (f x)))))))
4: ((λn.(λf.(λx.(((n (λg.(λh.(h (g f))))) (λu.x)) (λu.u))))) (λf.(λx.(f (f (f (f x)))))))
5: (λf.(λx.((((λf.(λx.(f (f (f (f x)))))) (λg.(λh.(h (g f))))) (λu.x)) (λu.u))))
6: (λf.(λx.(((λx.((λg.(λh.(h (g f)))) ((λg.(λh.(h (g f)))) ((λg.(λh.(h (g f)))) ((λg.(λh.(h (g f)))) x))))) (λu.x)) (λu.u))))
7: (λf.(λx.(((λg.(λh.(h (g f)))) ((λg.(λh.(h (g f)))) ((λg.(λh.(h (g f)))) ((λg.(λh.(h (g f)))) (λu.x))))) (λu.u))))
8: (λf.(λx.((λh.(h (((λg.(λh.(h (g f)))) ((λg.(λh.(h (g f)))) ((λg.(λh.(h (g f)))) (λu.x)))) f))) (λu.u))))
9: (λf.(λx.((λu.u) (((λg.(λh

# <span style="color: #A1F99C">9.</span>
<span style="color:#bb91f3">(SUBTRACT 3 5)</span> == <span style="color:#f1bd47">0</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [17]:
def term_test_9():
    return multi_app_term(arithmetic.SUBTRACT, arithmetic.number(3), arithmetic.number(5))


test_procedure(term_test_9())

0: (((λm.(λn.((n (λn.(λf.(λx.(((n (λg.(λh.(h (g f))))) (λu.x)) (λu.u)))))) m))) (λf.(λx.(f (f (f x)))))) (λf.(λx.(f (f (f (f (f x))))))))
1: ((λn.((n (λn.(λf.(λx.(((n (λg.(λh.(h (g f))))) (λu.x)) (λu.u)))))) (λf.(λx.(f (f (f x))))))) (λf.(λx.(f (f (f (f (f x))))))))
2: (((λf.(λx.(f (f (f (f (f x))))))) (λn.(λf.(λx.(((n (λg.(λh.(h (g f))))) (λu.x)) (λu.u)))))) (λf.(λx.(f (f (f x))))))
3: ((λx.((λn.(λf.(λx.(((n (λg.(λh.(h (g f))))) (λu.x)) (λu.u))))) ((λn.(λf.(λx.(((n (λg.(λh.(h (g f))))) (λu.x)) (λu.u))))) ((λn.(λf.(λx.(((n (λg.(λh.(h (g f))))) (λu.x)) (λu.u))))) ((λn.(λf.(λx.(((n (λg.(λh.(h (g f))))) (λu.x)) (λu.u))))) ((λn.(λf.(λx.(((n (λg.(λh.(h (g f))))) (λu.x)) (λu.u))))) x)))))) (λf.(λx.(f (f (f x))))))
4: ((λn.(λf.(λx.(((n (λg.(λh.(h (g f))))) (λu.x)) (λu.u))))) ((λn.(λf.(λx.(((n (λg.(λh.(h (g f))))) (λu.x)) (λu.u))))) ((λn.(λf.(λx.(((n (λg.(λh.(h (g f))))) (λu.x)) (λu.u))))) ((λn.(λf.(λx.(((n (λg.(λh.(h (g f))))) (λu.x)) (λu.u))))) ((λn.(λf.(λx.(((n (λg.(λh.(h (g f))))) (λu.x)) 

# <span style="color: #A1F99C">10.</span>
<span style="color:#bb91f3">(LEQ 3 2)</span> == <span style="color:#f1bd47">FALSE</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [18]:
def term_test_10():
    return multi_app_term(leq_term(), arithmetic.number(3), arithmetic.number(2))


test_procedure(term_test_10())

0: (((λmmm.(λnnn.((λn.((n (λx.(λx.(λy.y)))) (λx.(λy.x)))) (((λm.(λn.((n (λn.(λf.(λx.(((n (λg.(λh.(h (g f))))) (λu.x)) (λu.u)))))) m))) mmm) nnn)))) (λf.(λx.(f (f (f x)))))) (λf.(λx.(f (f x)))))
1: ((λnnn.((λn.((n (λx.(λx.(λy.y)))) (λx.(λy.x)))) (((λm.(λn.((n (λn.(λf.(λx.(((n (λg.(λh.(h (g f))))) (λu.x)) (λu.u)))))) m))) (λf.(λx.(f (f (f x)))))) nnn))) (λf.(λx.(f (f x)))))
2: ((λn.((n (λx.(λx.(λy.y)))) (λx.(λy.x)))) (((λm.(λn.((n (λn.(λf.(λx.(((n (λg.(λh.(h (g f))))) (λu.x)) (λu.u)))))) m))) (λf.(λx.(f (f (f x)))))) (λf.(λx.(f (f x))))))
3: (((((λm.(λn.((n (λn.(λf.(λx.(((n (λg.(λh.(h (g f))))) (λu.x)) (λu.u)))))) m))) (λf.(λx.(f (f (f x)))))) (λf.(λx.(f (f x))))) (λx.(λx.(λy.y)))) (λx.(λy.x)))
4: ((((λn.((n (λn.(λf.(λx.(((n (λg.(λh.(h (g f))))) (λu.x)) (λu.u)))))) (λf.(λx.(f (f (f x))))))) (λf.(λx.(f (f x))))) (λx.(λx.(λy.y)))) (λx.(λy.x)))
5: (((((λf.(λx.(f (f x)))) (λn.(λf.(λx.(((n (λg.(λh.(h (g f))))) (λu.x)) (λu.u)))))) (λf.(λx.(f (f (f x)))))) (λx.(λx.(λy.y)))) (λx.(λy.x)))
6: ((((

# <span style="color: #A1F99C">11.</span>
<span style="color:#bb91f3">(LEQ 2 5)</span> == <span style="color:#f1bd47">TRUE</span> == <span style="color:#f1bd47">λx. λy. x</span>

In [19]:
def term_test_11():
    return multi_app_term(leq_term(), arithmetic.number(2), arithmetic.number(5))


test_procedure(term_test_11())

0: (((λmmm.(λnnn.((λn.((n (λx.(λx.(λy.y)))) (λx.(λy.x)))) (((λm.(λn.((n (λn.(λf.(λx.(((n (λg.(λh.(h (g f))))) (λu.x)) (λu.u)))))) m))) mmm) nnn)))) (λf.(λx.(f (f x))))) (λf.(λx.(f (f (f (f (f x))))))))
1: ((λnnn.((λn.((n (λx.(λx.(λy.y)))) (λx.(λy.x)))) (((λm.(λn.((n (λn.(λf.(λx.(((n (λg.(λh.(h (g f))))) (λu.x)) (λu.u)))))) m))) (λf.(λx.(f (f x))))) nnn))) (λf.(λx.(f (f (f (f (f x))))))))
2: ((λn.((n (λx.(λx.(λy.y)))) (λx.(λy.x)))) (((λm.(λn.((n (λn.(λf.(λx.(((n (λg.(λh.(h (g f))))) (λu.x)) (λu.u)))))) m))) (λf.(λx.(f (f x))))) (λf.(λx.(f (f (f (f (f x)))))))))
3: (((((λm.(λn.((n (λn.(λf.(λx.(((n (λg.(λh.(h (g f))))) (λu.x)) (λu.u)))))) m))) (λf.(λx.(f (f x))))) (λf.(λx.(f (f (f (f (f x)))))))) (λx.(λx.(λy.y)))) (λx.(λy.x)))
4: ((((λn.((n (λn.(λf.(λx.(((n (λg.(λh.(h (g f))))) (λu.x)) (λu.u)))))) (λf.(λx.(f (f x)))))) (λf.(λx.(f (f (f (f (f x)))))))) (λx.(λx.(λy.y)))) (λx.(λy.x)))
5: (((((λf.(λx.(f (f (f (f (f x))))))) (λn.(λf.(λx.(((n (λg.(λh.(h (g f))))) (λu.x)) (λu.u)))))) (λf.(λx.(f 

# <span style="color: #A1F99C">12.</span>
<span style="color:#bb91f3">(EQ 2 2)</span> == <span style="color:#f1bd47">TRUE</span> == <span style="color:#f1bd47">λx. λy. x</span>

In [20]:
def term_test_12():
    return multi_app_term(eq_term(), arithmetic.number(2), arithmetic.number(2))


test_procedure(term_test_12())

0: (((λmm.(λnn.(((λp.(λq.((p q) p))) (((λmmm.(λnnn.((λn.((n (λx.(λx.(λy.y)))) (λx.(λy.x)))) (((λm.(λn.((n (λn.(λf.(λx.(((n (λg.(λh.(h (g f))))) (λu.x)) (λu.u)))))) m))) mmm) nnn)))) mm) nn)) (((λmmm.(λnnn.((λn.((n (λx.(λx.(λy.y)))) (λx.(λy.x)))) (((λm.(λn.((n (λn.(λf.(λx.(((n (λg.(λh.(h (g f))))) (λu.x)) (λu.u)))))) m))) mmm) nnn)))) nn) mm)))) (λf.(λx.(f (f x))))) (λf.(λx.(f (f x)))))
1: ((λnn.(((λp.(λq.((p q) p))) (((λmmm.(λnnn.((λn.((n (λx.(λx.(λy.y)))) (λx.(λy.x)))) (((λm.(λn.((n (λn.(λf.(λx.(((n (λg.(λh.(h (g f))))) (λu.x)) (λu.u)))))) m))) mmm) nnn)))) (λf.(λx.(f (f x))))) nn)) (((λmmm.(λnnn.((λn.((n (λx.(λx.(λy.y)))) (λx.(λy.x)))) (((λm.(λn.((n (λn.(λf.(λx.(((n (λg.(λh.(h (g f))))) (λu.x)) (λu.u)))))) m))) mmm) nnn)))) nn) (λf.(λx.(f (f x))))))) (λf.(λx.(f (f x)))))
2: (((λp.(λq.((p q) p))) (((λmmm.(λnnn.((λn.((n (λx.(λx.(λy.y)))) (λx.(λy.x)))) (((λm.(λn.((n (λn.(λf.(λx.(((n (λg.(λh.(h (g f))))) (λu.x)) (λu.u)))))) m))) mmm) nnn)))) (λf.(λx.(f (f x))))) (λf.(λx.(f (f x)))))) (((

# <span style="color: #A1F99C">13.</span>
<span style="color:#bb91f3">(EQ 1 4)</span> == <span style="color:#f1bd47">FALSE</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [21]:
def term_test_13():
    return multi_app_term(eq_term(), arithmetic.number(1), arithmetic.number(4))


test_procedure(term_test_13())

0: (((λmm.(λnn.(((λp.(λq.((p q) p))) (((λmmm.(λnnn.((λn.((n (λx.(λx.(λy.y)))) (λx.(λy.x)))) (((λm.(λn.((n (λn.(λf.(λx.(((n (λg.(λh.(h (g f))))) (λu.x)) (λu.u)))))) m))) mmm) nnn)))) mm) nn)) (((λmmm.(λnnn.((λn.((n (λx.(λx.(λy.y)))) (λx.(λy.x)))) (((λm.(λn.((n (λn.(λf.(λx.(((n (λg.(λh.(h (g f))))) (λu.x)) (λu.u)))))) m))) mmm) nnn)))) nn) mm)))) (λf.(λx.(f x)))) (λf.(λx.(f (f (f (f x)))))))
1: ((λnn.(((λp.(λq.((p q) p))) (((λmmm.(λnnn.((λn.((n (λx.(λx.(λy.y)))) (λx.(λy.x)))) (((λm.(λn.((n (λn.(λf.(λx.(((n (λg.(λh.(h (g f))))) (λu.x)) (λu.u)))))) m))) mmm) nnn)))) (λf.(λx.(f x)))) nn)) (((λmmm.(λnnn.((λn.((n (λx.(λx.(λy.y)))) (λx.(λy.x)))) (((λm.(λn.((n (λn.(λf.(λx.(((n (λg.(λh.(h (g f))))) (λu.x)) (λu.u)))))) m))) mmm) nnn)))) nn) (λf.(λx.(f x)))))) (λf.(λx.(f (f (f (f x)))))))
2: (((λp.(λq.((p q) p))) (((λmmm.(λnnn.((λn.((n (λx.(λx.(λy.y)))) (λx.(λy.x)))) (((λm.(λn.((n (λn.(λf.(λx.(((n (λg.(λh.(h (g f))))) (λu.x)) (λu.u)))))) m))) mmm) nnn)))) (λf.(λx.(f x)))) (λf.(λx.(f (f (f (f x))))